# 正向化函数组

In [1]:

# 极小型指标转化为极大型指标的函数
def minTomax(maxx, x):
    x = list(x)  # 将输入的指标数据转换为列表
    ans = [[(maxx-e)] for e in x]  # 计算最大值与每个指标值的差，并将其放入新列表中
    return np.array(ans)  # 将列表转换为numpy数组并返回

# 中间型指标转化为极大型指标的函数
def midTomax(bestx, x):
    x = list(x)  # 将输入的指标数据转换为列表
    h = [abs(e-bestx) for e in x]  # 计算每个指标值与最优值之间的绝对差
    M = max(h)  # 找到最大的差值
    if M == 0:
        M = 1  # 防止最大差值为0的情况
    ans = [[(1-e/M)] for e in h]  # 计算每个差值占最大差值的比例，并从1中减去，得到新指标值
    return np.array(ans)  # 返回处理后的numpy数组

# 区间型指标转化为极大型指标的函数
def regTomax(lowx, highx, x):
    x = list(x)  # 将输入的指标数据转换为列表
    M = max(lowx-min(x), max(x)-highx)  # 计算指标值超出区间的最大距离
    if M == 0:
        M = 1  # 防止最大距离为0的情况
    ans = []
    for i in range(len(x)):
        if x[i]<lowx:
            ans.append([(1-(lowx-x[i])/M)])  # 如果指标值小于下限，则计算其与下限的距离比例
        elif x[i]>highx:
            ans.append([(1-(x[i]-highx)/M)])  # 如果指标值大于上限，则计算其与上限的距离比例
        else:
            ans.append([1])  # 如果指标值在区间内，则直接取为1
    return np.array(ans)  # 返回处理后的numpy数组


# 初始化数据

In [2]:
import numpy as np
init_matrix = [[9,10,175,120], [8,7,164,80],[6,3,157,90]]
init_matrix = np.array(init_matrix)
objectNum = len(init_matrix)
indicatorNum = len(init_matrix[0])
kind = [1,2,3,4]

# 指标类型，按顺序输入，1:极大型，2：极小型，3：中间型，4：区间型
kind = [1,2,3,4]
# 中间值的最优值
bestA = 165
# 区间型的最大值和最小值
lowA = 90
highA = 100

# Step1. 构建子母序列

## 数据正向化

In [3]:
X = np.zeros(shape=(objectNum, 0))  # 初始化为 0 列，因为后续会逐列拼接

for i in range(indicatorNum):
    if kind[i] == 1:  # 如果当前指标为极大型，则直接使用原值
        v = np.array(init_matrix[:, i])
    elif kind[i] == 2:  # 如果当前指标为极小型，调用 minTomax 函数转换
        maxA = max(init_matrix[:, i])
        v = minTomax(maxA, init_matrix[:, i])
    elif kind[i] == 3:  # 如果当前指标为中间型，调用 midTomax 函数转换
        v = midTomax(bestA, init_matrix[:, i])
    elif kind[i] == 4:  # 如果当前指标为区间型，调用 regTomax 函数转换
        v = regTomax(lowA, highA, init_matrix[:, i])
    
    if i==0:
        X = v.reshape(-1, 1)
    else:
        X = np.hstack([X, v])  # 否则，将新指标列拼接到 X 数组上

print("统一指标后矩阵为：\n{}".format(X))  # 打印处理后的矩阵 X

统一指标后矩阵为：
[[9.  0.  0.  0. ]
 [8.  3.  0.9 0.5]
 [6.  7.  0.2 1. ]]


## 数据预处理

In [4]:
Mean = np.mean(X, axis=0)
ripeMatrix = X / Mean
print('预处理后的矩阵为：')
print(ripeMatrix)

预处理后的矩阵为：
[[1.17391304 0.         0.         0.        ]
 [1.04347826 0.9        2.45454545 1.        ]
 [0.7826087  2.1        0.54545455 2.        ]]


## 构建子母序列

In [5]:
Y = np.max(ripeMatrix, axis=1)  # 母序列为虚拟的，用每一行的最大值构成的列向量表示母序列

# Step2. 计算灰色关联系数

In [12]:
absX0_Xi = np.abs(ripeMatrix - np.tile(Y.reshape(-1, 1), (1, ripeMatrix.shape[1])))
a = np.min(absX0_Xi)
b = np.max(absX0_Xi)
# 分辨系数取0.5
rho = 0.5  
# 计算子序列中各个指标与母序列的关联系数
gamma = (a + rho * b) / (absX0_Xi + rho * b)  
print("a = ",a,"b = ", b)
print("关联系数矩阵")
print(gamma)

关联系数：
a =  0.0 b =  1.5545454545454547
关联系数矩阵
[[1.         0.39835916 0.39835916 0.39835916]
 [0.3551883  0.33333333 1.         0.34826884]
 [0.37107274 1.         0.33333333 0.88601036]]


# Step3. 计算关联度，权重，得分

## 通过关联度计算权重

In [13]:
correlationCoefficient = np.mean(gamma, axis=0)
weight = correlationCoefficient / np.sum(np.mean(gamma, axis=0))  # 利用子序列中各个指标的灰色关联度计算权重
print("correlationCoefficient = ", correlationCoefficient)
print(weight)

correlationCoefficient =  [0.57542035 0.57723083 0.57723083 0.54421279]
[0.2530327  0.25382883 0.25382883 0.23930963]


## 计算综合得分

In [14]:
score = np.sum(X * np.tile(weight, (X.shape[0], 1)), axis=1)   # 未归一化的得分
stand_S = score / np.sum(score)  # 归一化后的得分
sorted_S = np.sort(stand_S)[::-1]  # 进行降序排序
index = np.argsort(stand_S)[::-1]  # 排序后的索引

print('归一化后的得分及其索引（降序）：')
print(sorted_S)
print(index)

归一化后的得分及其索引（降序）：
[0.39850902 0.34835187 0.25313911]
[2 1 0]
